# Links 
- https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html
- https://medium.com/@abinovarghese/installing-coco-api-in-windows-python-9b4dfc3812ef

# Installation

## Nb_extensions
For instance for table of content and others
- pip install jupyter_contrib_nbextensions
- jupyter contrib nbextension install --user

## Tensor Flow Object Detection API TF 2 (PIP)
WICHTIG: Die aktuellen TF Version hat nicht funktioniert! Ein pip install tensorflow hat das Problem behoben.
- git clone https://github.com/tensorflow/models.git in ein Verzeichnis mit dem Namen TensorFlow. Es sollte dann TensorFlow\models\research existieren
- falls noch nicht verhanden, google protobuf herunter laden
    - neueste zip version https://github.com/protocolbuffers/protobuf/releases auswählen (windows Version) und in einen Ordner extrahieren
    - Pfad anpassen in den Umgebungsvariablen und Pfad zu protobuf hinzufügen (den Pfad zum ...\bin)
    - cmd -> protoc sollte nun funktionieren
- kopiere ...\TensorFlow\models\research\object_detection\packages\tf2\setup.py nach ...\TensorFlow\models\research\setup.py
- gehe in den Ordner ...\TensorFlow\models\research\ und installiere das Packet mit python -m pip install .
    - Falls es hier zu einem Fehler kommt bei Running setup.py install for pycocotools ... error: ... dann:
        - pip install cython
        - pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI
            - Falls es zu einem Fehler kommt: Das System kann den angegebenen Pfad nicht finden: 'C:\\Program Files (x86)\\Microsoft Visual Studio 14.0\\VC\\PlatformSDK\\lib'
                - VS 19 Build Tools installieren ( https://visualstudio.microsoft.com/thank-you-downloading-visual-studio/?sku=BuildTools&rel=16 )
                
## Tensor Flow Object Detection API TF 2 (Versuch Anaconda)

- Anaconda herunterladen
- Tensorflow, Keras auswählen im Navigator
- git clone  TensorFlow/models/research/ https://github.com/tensorflow/models.git in ein Verzeichnis mit dem Namen TensorFlow
- cd TensorFlow\models\research 
- protoc object_detection/protos/*.proto --python_out=. 
- evt. falls später ein Fehler kommt bei pycocotools: pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI
    - Hier gab es ein Problem mit dem C++ kompiler, VS 19 Build Tools installieren hat das Problem gelöst 
- copy object_detection/packages/tf2/setup.py . (evt. händisch kopieren)
- python -m pip install .
- testen mit python object_detection/builders/model_builder_tf2_test.py

## Tensor Flow Object Detection API TF 2 (PIP, 2. Versuch)
Die Lösung war die build tools von Visual Studio 2019 zu installieren, damit war pycoco über philferriere's Repository kompilierbar und die Installation hat geklappt

## Ausführen auf der Grafikkarte
Es wird für die Installation von CUDA eine aktuelle Installation von Visual Studio benötigt, VS 15 wird nicht mehr unterstützt. Dafür habe ich die Community Edition 2019 herunter geladen und hier die C++ Entwicklungsoptionen aktiviert. Besonders wichtig ist der MSVC Compiler. Die benötigten Versionen finden sich hier (https://docs.nvidia.com/cuda/cuda-installation-guide-microsoft-windows/index.html)

Danach die cuda toolskit Version 11.* (ich hatte 11.3) installieren (https://developer.nvidia.com/cuda-downloads?target_os=Windows&target_arch=x86_64&target_version=10&target_type=exe_local)

Und ebenso die zugehörige cudNN (https://developer.nvidia.com/rdp/cudnn-download) und über die Anleitung https://docs.nvidia.com/deeplearning/cudnn/install-guide/index.html#install-windows entpacken und im Cuda Ordner platzieren (standard ist C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.3)

Das skript TensorFlow\scripts\gpu_test.py sollte nun die Grafikkarte finden (standardname ist 0)

## Andere Tools
### LabelImg
- Zum einfachen annotieren von Daten, herunterladen über pip install labelImg, aufrufen über labelImg in der Konsole

# Daten Vorbereiten

- Im Ordner TensorFlow\workspace einen neuen Projektordner erstellen (workspace evt neu erstellen)
- Bilddaten in ein passendes Verzeichnis kopieren und einen annotations Ordner anlegen
- Bilddaten annotieren (zB labelImg), diese annotationen werden meist als .xml Datei gespeichert und sollen im gleichen Ordner liegen
- Bilder aufteilen in test und trainingsdaten, dafür das skript in TensorFlow\scripts\preprocessing verwenden mit python partition_dataset.py [-h] [-i IMAGEDIR] [-o OUTPUTDIR] [-r RATIO] [-x] , wobei -x gesetzt werden muss um die .xml Datein mit zu kopieren
- Als nächstes muss die Labelmap erstellt werden, diese weist jedem label des Netzes eine ID zu und hat die Dateiendung .pbtxt. Diese kommt in den annotations Ordner bsp:

- Tensorflow erwartet für die Annotationen das TFRecord Format, dieses kann über:
    - TensorFlow\scripts\preprocessing\generate_tfrecord.py [-h] [-x XML_DIR] [-l LABELS_PATH] [-o OUTPUT_PATH] [-i IMAGE_DIR] [-c CSV_PATH] erstellt werden

# Model vorbereiten

- Die gewünschten vortrainierten Netze können von https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md herunter geladen werde. Diese dann entpacken (2mal) und in einen Ordner pre-trained-models im Projektverzeichnis kopieren. 
- Die pipeline.config Datei aus dem vortrainierten Netzt nun in den leeren Ordner workspace\Projektname\models kopieren
- In dieser müssen nun die Werte angepasst werden: !ACHTUNG!: Alle Pfade müssen mit / getrennt werden und nicht mit \, sonst kommt ein UnicodeDecoder Fehler
    - num_class (Anzahl an zu erkennenden Klassen)
    - batch_size (größer bei mehr verfügbarem Speicher)
    - fine_tune_checkpoint: "..." (Pfad zu checkpoint ../checkpoint/ckpt-0)
    - fine_tune_checkpoint_type: "detection" ("classification wenn nur die klassifizierung geändert werden soll)
    - use_bfloat16: false (nur true wenn auf einem TPU trainiert wird)
    - label_map_path: "annotations/label_map.pbtxt" (Pfad zur label map Datei, in train_input_reader und eval_input_reader)
    - train_input_reader { ... input_path: "annotations/train.record" (Pfad zur train .record Datei)
    - eval_config { 
        - metrics_set: "coco_detection_metrics"
        - use_moving_averages: false (Für die Evaluierung des Models, beide Einstellungen sind optional)
    - eval_input_reader { ...input_path: "annotations/test.record" (Pfad zur test .record Datei)
    - kopiere ..\TensorFlow\models\research\object_detection\model_main_tf2.py

### weiterführende anpassungen
unter https://github.com/tensorflow/models kann über die Suche von parameter_name path:research/object_detection/protos , wobei parameter_name natürlich ausgetauscht werden muss, näheres über die möglichen Parameter gefunden werden, die die ODA annimmt.

# Training

In [ ]:
- python model_main_tf2.py --model_dir=models/ssd_resnet --pipeline_config_path=models/ssd_resnet/pipeline.config checkpoint_every_n=1500

## Monitoring
Das Training kann über Tensorboard beobachtet werden, dafür eine neue Kommandozeile öffnen und tensorboard starten (im Projektordner)
- tensorboard --logdir=models/ssd_resnet

## Evaluierung
Um die Metriken zu berechnen muss neben dem Training ein zweiter Prozess gestartet werden, dafür eine neue cmd öffnen, aktivieren und ins workspace verzeichnis wechseln. Dort dann den befehl (siehe unten) ausführen. Wichtig hierbei ist, dass der checkpoint pfad gefunden wird, ansonsten wartet er endlos auf einen neuen Checkpoint. Hierfür hat ein ./ vor dem Pfad geholfen, um zu zeigen dass das Lokale Verzeichnis der Startpunkt ist.


In [ ]:
python model_main_tf2.py --pipeline_config_path=models/ssd_resnet/pipeline.config --model_dir=models/ssd_resnet --checkpoint_dir=./models/ssd_resnet/ --num_workers=2 --sample_1_of_n_eval_examples=1

Pycoco Fehler mit numpy linspace
In Datei C:\sma_2.0.1.dev3\python-3.7.6.amd64\Lib\site-packages\pycocotools Zeile 507 und 507 zu:

In [ ]:
self.iouThrs = np.linspace(.5, 0.95, int(np.round((0.95 - .5) / .05)) + 1, endpoint=True)
self.recThrs = np.linspace(.0, 1.00, int(np.round((1.00 - .0) / .01)) + 1, endpoint=True)

ändern

## Model speichern
- TensorFlow/models/research/object_detection/exporter_main_v2.py ins Projektverzeichnis kopieren
- Dieses dann ausführen mit:
    - 

In [ ]:
python exporter_main_v2.py --input_type image_tensor --pipeline_config_path models\ssd_resnet\pipeline.config --trained_checkpoint_dir models\ssd_resnet\ --output_directory exported_models\ssd_resnet

## Ergebnisse laden und anwenden
https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/auto_examples/plot_object_detection_saved_model.html

https://towardsdatascience.com/custom-object-detection-using-tensorflow-from-scratch-e61da2e10087